In [2]:
pip install rdflib

     |████████████████████████████████| 379 kB 4.9 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 


xmlファイルを読み取る

In [3]:
import xml.etree.ElementTree as ET
with open('/content/Step1.5の記述.xml', 'r', encoding = 'utf-8') as f:
  tree = ET.parse(f)

root = tree.getroot()

In [10]:
from rdflib import Graph, URIRef, Literal, BNode, Namespace
from rdflib.namespace import FOAF, RDF, OWL, RDFS, SKOS, DCTERMS

data = Graph()

pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')
pd3ex1 = Namespace('http://DigitalTriplet.net/2021/08/ex1#')

data.bind('pd3', pd3)
data.bind('pd3ex1', pd3ex1)
data.bind('rdf', RDF)

#t =0
for mxCell in root.iter('mxCell'):
  # t = t+1
  # if(t>50):
  #   break
  style = mxCell.get('style')
  if(style != None):

    #action
    if('pd3type=action' in style):
      action = URIRef('http://DigitalTriplet.net/2021/08/ex1#' + mxCell.get('id'))
      #idとvalueを取得
      id = mxCell.get('id')
      value = mxCell.get('value')

      data.add((action, RDF.type, pd3.Action))
      data.add((action, pd3.id, Literal(id)))
      data.add((action, pd3.value, Literal(value)))

      #座標、形状を取得
      data.add((action, pd3.geoBoundingWidth, Literal(mxCell[0].get('width'))))
      data.add((action, pd3.geoBoundingHeight, Literal(mxCell[0].get('height'))))
      data.add((action, pd3.geoBoundingX, Literal(mxCell[0].get('x'))))
      data.add((action, pd3.geoBoundingY, Literal(mxCell[0].get('y'))))

      pd3actioncheck = True
      for element in style.split(';'):
        if('pd3layer=' in element):
          layer = Literal(element.replace('pd3layer=', ''))
          data.add((action, pd3.layer, layer))
        elif('pd3action=' in element):
          actionType = Literal(element.replace('pd3action=', ''))
          data.add((action, pd3.actionType, actionType))
          pd3actioncheck = False
      if(pd3actioncheck):
        data.add((action,pd3.actionType, Literal('nil')))
      pd3actioncheck = True

      #attribution, container, input, outputを取得
      attribution_id = mxCell.get('parent')

      for mxCell1 in root.iter('mxCell'):
        if(mxCell1.get('target') == id):
          data.add((action, pd3.input, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + mxCell1.get('id'))))
          source_id = mxCell1.get('source')
          for mxCell2 in root.iter('mxCell'):
            if((mxCell2.get('style') != None) & (mxCell2.get('id') == source_id)):
              if('pd3type=container' in mxCell2.get('style')):
                data.add((action, pd3.expansion, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + mxCell2.get('id'))))
                break
        elif((mxCell1.get('id') == attribution_id) & (mxCell1.get('style') != None)):
          data.add((action, pd3.attribution, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + attribution_id)))
        elif(mxCell1.get('source') == id):
          data.add((action, pd3.output, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + mxCell1.get('id'))))

    #container
    elif('pd3type=container' in style):
      container = URIRef('http://DigitalTriplet.net/2021/08/ex1#' + mxCell.get('id'))
      #idを取得
      id = mxCell.get('id')

      data.add((container, RDF.type, pd3.Container))
      data.add((container, pd3.id, Literal(id)))
      

      #座標、形状を取得
      data.add((container, pd3.geoBoundingWidth, Literal(mxCell[0].get('width'))))
      data.add((container, pd3.geoBoundingHeight, Literal(mxCell[0].get('height'))))
      data.add((container, pd3.geoBoundingX, Literal(mxCell[0].get('x'))))
      data.add((container, pd3.geoBoundingY, Literal(mxCell[0].get('y'))))

      #layer, containerTypeを取得
      for element in style.split(';'):
        if('pd3layer=' in element):
          layer = Literal(element.replace('pd3layer=', ''))
          data.add((container, pd3.layer, layer))
        elif('containertype=' in element):
          containertype = element.replace('containertype=', '')
          data.add((container, pd3.containerType, Literal(containertype)))
          
      #valueを取得
      if(containertype == 'whilebox' or containertype == 'whilecontainer'):
        value = 'nil'
      else:
        value = mxCell.get('value')
      data.add((container, pd3.value, Literal(value)))

      #member, target, source, contractionを取得
      for mxCell1 in root.iter('mxCell'):
        if(mxCell1.get('parent') == id):
          data.add((container, pd3.member, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + mxCell1.get('id'))))
        elif(mxCell1.get('source') == id):
          target_id = mxCell1.get('target')
          data.add((container, pd3.output, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + target_id)))
          for mxCell2 in root.iter('mxCell'):
            if((mxCell2.get('style') != None) & (mxCell2.get('id') == target_id)):
              if('pd3type=action' in mxCell2.get('style')):
                data.add((container, pd3.contraction, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + mxCell2.get('id'))))
                break


  #arc
    elif('pd3type=arrow' in style):
      if(mxCell.get('target') != None):
        arc = URIRef('http://DigitalTriplet.net/2021/08/ex1#' + mxCell.get('id'))

        #id,valueを取得
        id = mxCell.get('id')
        value = mxCell.get('value')
        if(value ==''):
          for mxCell1 in root.iter('mxCell'):
              if(mxCell1.get('style') != None):
                if('edgeLabel' in mxCell1.get('style')):
                  if(id == mxCell1.get('parent')):
                    value = mxCell1.get('value')
                    break
        elif(value == None):
          value = ""
        data.add((arc, pd3.id, Literal(id)))
        data.add((arc, pd3.value, Literal(value)))

        #layerを取得
        for element in style.split(';'):
          if('pd3layer=' in element):
            layer = element.replace('pd3layer=', '')
        data.add((arc, pd3.layer, Literal(layer)))

        

        #sourceがあればFlowかContainerFlow
        source = mxCell.get('source')
        target = mxCell.get('target')
        if(source != None):
          #source, targetを取得
          data.add((arc, pd3.source, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + source)))
          data.add((arc, pd3.target, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + target)))

          #exitX, exitY, entryX, entryYを取得
          
          for element in style.split(';'):
            if('exitX=' in element):
              exitX = element.replace('exitX=', '')
              data.add((arc, pd3.exitX, Literal(exitX)))
            elif('exitY=' in element):
              exitY = element.replace('exitY=', '')
              data.add((arc, pd3.exitY, Literal(exitY)))
            elif('entryX=' in element):
              entryX = element.replace('entryX=', '')
              data.add((arc, pd3.entryX, Literal(entryX)))
            elif('entryY=' in element):
              entryY = element.replace('entryY=', '')
              data.add((arc, pd3.entryY, Literal(entryY)))
                
          for mxCell1 in root.iter('mxCell'):
            if(mxCell1.get('id') == source):
              #Flow
              if(mxCell1.get('style') != None):
                if('pd3type=action' in mxCell1.get('style')):
                    data.add((arc, RDF.type, pd3.Flow))
                    data.add((arc, pd3.arcType, Literal('Flow')))
                    break

                  #ContainerFlow
                elif('pd3type=container' in mxCell1.get('style')):
                  data.add((arc, RDF.type, pd3.ContainerFlow))
                  data.add((arc, pd3.arcType, Literal('ContainerFlow')))
                  break

        else:
          #sourceがなければSupFlow
          data.add((arc, RDF.type, pd3.SupFlow))

          #targetの取得
          data.add((arc, pd3.target, URIRef('http://DigitalTriplet.net/2021/08/ex1#' + target)))

          #entryX,entryYの取得
          for element in style.split(';'):
            if('entryX=' in element):
              entryX = element.replace('entryX=', '')
              data.add((arc, pd3.entryX, Literal(entryX)))
            elif('entryY=' in element):
              entryY = element.replace('entryY=', '')
              data.add((arc, pd3.entryY, Literal(entryY)))

          #arcTypeの取得
          if("entryY=1;" in style):
            data.add((arc, pd3.arcType, Literal('tool/knowledge')))
          elif("entryY=0.5;" in style):
            data.add((arc, pd3.arcType, Literal('rationale')))
          elif("entryY=0;" in style):
            if("entryX=0.5;" in style):
              data.add((arc, pd3.arcType, Literal('intention')))
            elif("entryX=1;" in style):
              data.add((arc, pd3.arcType, Literal('annotation')))

          #位置を取得
          for mxPoint in mxCell.iter('mxPoint'):
            if(mxPoint.get('as') == 'sourcePoint'):
              data.add((arc, pd3.geoSourcePointX, Literal(mxPoint.get('x'))))
              data.add((arc, pd3.geoSourcePointY, Literal(mxPoint.get('y'))))
          
  
       
#print(data.serialize())

with open('/content/Step1.5の記述ver2.ttl', 'w', encoding = 'utf-8') as f:
  f.write(data.serialize())

@prefix pd3: <http://DigitalTriplet.net/2021/08/ontology#> .
@prefix pd3ex1: <http://DigitalTriplet.net/2021/08/ex1#> .

pd3ex1:WbIF7mzcT_pnHTd9FjXQ-16 a pd3:ContainerFlow ;
    pd3:arcType "ContainerFlow" ;
    pd3:entryX "0" ;
    pd3:entryY "1" ;
    pd3:id "WbIF7mzcT_pnHTd9FjXQ-16" ;
    pd3:layer "topic" ;
    pd3:source pd3ex1:WbIF7mzcT_pnHTd9FjXQ-17 ;
    pd3:target pd3ex1:WbIF7mzcT_pnHTd9FjXQ-11 ;
    pd3:value "" .

pd3ex1:WbIF7mzcT_pnHTd9FjXQ-35 a pd3:Action ;
    pd3:actionType "end" ;
    pd3:geoBoundingHeight "60" ;
    pd3:geoBoundingWidth "60" ;
    pd3:geoBoundingX "2038" ;
    pd3:geoBoundingY "166" ;
    pd3:id "WbIF7mzcT_pnHTd9FjXQ-35" ;
    pd3:input pd3ex1:WbIF7mzcT_pnHTd9FjXQ-32 ;
    pd3:layer "topic" ;
    pd3:value "End" .

pd3ex1:WbIF7mzcT_pnHTd9FjXQ-7 a pd3:Action ;
    pd3:actionType "start" ;
    pd3:geoBoundingHeight "60" ;
    pd3:geoBoundingWidth "60" ;
    pd3:geoBoundingX "60" ;
    pd3:geoBoundingY "166" ;
    pd3:id "WbIF7mzcT_pnHTd9FjXQ-7" ;
    pd3